### THE GENERATED CODE BY LLM

In [7]:
from z3 import *

def z3_to_float(z3_val):
    """
    將已經經過模型求解的 Z3 數值轉換成 float。
    優先使用 as_decimal() 方法，如果返回字串以 '?' 結尾則移除，否則直接轉換。
    """
    try:
        dec_str = z3_val.as_decimal(10)
        if dec_str.endswith('?'):
            dec_str = dec_str[:-1]
        return float(dec_str)
    except Exception:
        return float(str(z3_val))

def calculate_securities_tax_item(tax_item, tp, ep=None, sc=None):
    """
    計算證券交易稅中的某一項目。

    參數：
      - tax_item: 必須為下列其中之一：
          "stock"                 : 公司發行之股票及表明股票權利憑證（成交價格×0.003）
          "bond"                  : 公司債及其他經政府核准之有價證券（成交價格×0.001）
          "warrant"               : 認購(售)權證（成交價格×0.001）
          "warrant_delivery_stock": 權證股票交割（履約價格×股數×0.003）
          "warrant_delivery_cash" : 權證現金結算（履約價格×股數×0.001）
      - tp: 成交價格（數值）
      - ep: 履約價格（僅對 warrant_delivery_stock 與 warrant_delivery_cash 有效）
      - sc: 股數（僅對 warrant_delivery_stock 與 warrant_delivery_cash 有效）

    回傳：以「整數+小數點兩位」格式的字串，例如 "300.25"，
           此處若 SMT 計算結果為 1916.129，則只顯示 1916.12（即截斷小數點後第三位）。
    """
    transaction_price = RealVal(tp)
    s = Solver()
    s.add(transaction_price == tp)
    
    if tax_item == "stock":
        # 稅率 0.003
        tax_expr = transaction_price * 0.003
    elif tax_item == "bond":
        # 稅率 0.001
        tax_expr = transaction_price * 0.001
    elif tax_item == "warrant":
        # 稅率 0.001
        tax_expr = transaction_price * 0.001
    elif tax_item == "warrant_delivery_stock":
        if ep is None or sc is None:
            raise ValueError("計算權證股票交割必須提供履約價格與股數")
        exercise_price = RealVal(ep)
        share_count = RealVal(sc)
        s.add(exercise_price == ep, share_count == sc)
        tax_expr = exercise_price * share_count * 0.003
    elif tax_item == "warrant_delivery_cash":
        if ep is None or sc is None:
            raise ValueError("計算權證現金結算必須提供履約價格與股數")
        exercise_price = RealVal(ep)
        share_count = RealVal(sc)
        s.add(exercise_price == ep, share_count == sc)
        tax_expr = exercise_price * share_count * 0.001
    else:
        raise ValueError("未知的證券交易稅項目")
    
    if s.check() == sat:
        m = s.model()
        tax_val = m.evaluate(tax_expr, model_completion=True)
        # 轉成 float 並截斷到小數點後兩位 (非四捨五入)
        val = z3_to_float(tax_val)
        truncated = int(val * 100) / 100
        return f"{truncated:.2f}"
    else:
        return None

def calculate_futures_tax_item(tax_item, ca, pa=None):
    """
    計算期貨交易稅中的某一項目。

    參數：
      - tax_item: 必須為下列其中之一：
          "stock_index"      : 股價類期貨（契約金額×0.00002）
          "interest_rate_30" : 30天期商業本票利率期貨（契約金額×0.000000125）
          "interest_rate_10" : 10年期政府債期貨（契約金額×0.00000125）
          "option"           : 選擇權及期貨選擇權（權利金金額×0.001）
          "gold"             : 黃金期貨（契約金額×0.0000025）
      - ca: 契約金額（數值，除 "option" 外皆以此計算）
      - pa: 權利金金額（僅對 "option" 有效）

    回傳：以「整數+小數點兩位」格式的字串，例如 "100.25"，
           同樣會截斷超過小數點後第二位的數字。
    """
    contract_amount = RealVal(ca)
    s = Solver()
    s.add(contract_amount == ca)
    
    if tax_item == "stock_index":
        tax_expr = contract_amount * 0.00002
    elif tax_item == "interest_rate_30":
        tax_expr = contract_amount * 0.000000125
    elif tax_item == "interest_rate_10":
        tax_expr = contract_amount * 0.00000125
    elif tax_item == "option":
        if pa is None:
            raise ValueError("計算選擇權稅必須提供權利金金額")
        premium_amount = RealVal(pa)
        s.add(premium_amount == pa)
        tax_expr = premium_amount * 0.001
    elif tax_item == "gold":
        tax_expr = contract_amount * 0.0000025
    else:
        raise ValueError("未知的期貨交易稅項目")
    
    if s.check() == sat:
        m = s.model()
        tax_val = m.evaluate(tax_expr, model_completion=True)
        val = z3_to_float(tax_val)
        truncated = int(val * 100) / 100
        return f"{truncated:.2f}"
    else:
        return None

# 示範如何呼叫：
if __name__ == '__main__':
    # 證券交易稅測試範例
    sec_result = calculate_securities_tax_item("stock", 100000)
    print("證券交易稅_股票：", sec_result)
    
    sec_result2 = calculate_securities_tax_item("warrant_delivery_stock", 100000, ep=50, sc=1000)
    print("證券交易稅_權證股票交割：", sec_result2)
    
    # 期貨交易稅測試範例
    fut_result = calculate_futures_tax_item("option", 5000000, pa=20000)
    print("期貨交易稅_選擇權及期貨選擇權：", fut_result)


證券交易稅_股票： 300.00
證券交易稅_權證股票交割： 150.00
期貨交易稅_選擇權及期貨選擇權： 20.00


### Selenium Test

In [11]:
import random
import time
import re
from decimal import Decimal, ROUND_FLOOR

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import (
    TimeoutException,
    ElementClickInterceptedException,
    StaleElementReferenceException,
)

URL = "https://www.etax.nat.gov.tw/etwmain/etw158w/49"
TYPE_DELAY = 0.08


# =========================================================
# ✅ 只用 notebook 已定義好的本地函式（不 import）
# =========================================================
def _get_local_functions():
    g = globals()
    if "calculate_securities_tax_item" not in g:
        raise RuntimeError("你 notebook 目前沒有定義 calculate_securities_tax_item，請先跑你本地 Z3 code 的 cell。")
    if "calculate_futures_tax_item" not in g:
        raise RuntimeError("你 notebook 目前沒有定義 calculate_futures_tax_item，請先跑你本地 Z3 code 的 cell。")
    return g["calculate_securities_tax_item"], g["calculate_futures_tax_item"]


# =========================================================
# Selenium：通用
# =========================================================
def human_type(element, text, delay=TYPE_DELAY):
    for ch in str(text):
        element.send_keys(ch)
        time.sleep(delay)
    element.send_keys(Keys.TAB)


def safe_clear(driver, el):
    try:
        el.clear()
    except Exception:
        driver.execute_script("arguments[0].value='';", el)
    try:
        el.send_keys(Keys.TAB)
    except Exception:
        pass


def dismiss_gsc_overlay(driver):
    # 保險：避免 overlay 擋點擊（不一定會出現）
    try:
        close_btns = driver.find_elements(By.CSS_SELECTOR, ".gsc-results-close-btn")
        for b in close_btns:
            if b.is_displayed() and b.is_enabled():
                try:
                    b.click()
                    time.sleep(0.05)
                except Exception:
                    pass

        try:
            driver.find_element(By.TAG_NAME, "body").send_keys(Keys.ESCAPE)
            time.sleep(0.05)
        except Exception:
            pass

        overlays = driver.find_elements(By.CSS_SELECTOR, ".gsc-results-wrapper-overlay")
        for ov in overlays:
            if ov.is_displayed():
                driver.execute_script(
                    "arguments[0].style.display='none'; arguments[0].style.pointerEvents='none';",
                    ov,
                )
    except Exception:
        pass


def safe_click(driver, el):
    dismiss_gsc_overlay(driver)
    driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
    time.sleep(0.08)
    for _ in range(3):
        try:
            el.click()
            return
        except (ElementClickInterceptedException, StaleElementReferenceException):
            dismiss_gsc_overlay(driver)
            try:
                driver.execute_script("arguments[0].click();", el)
                return
            except Exception:
                time.sleep(0.15)
    raise RuntimeError("click 失敗（可能被遮罩層/動畫擋住）")


def _truncate_2_from_decimal(dec: Decimal) -> Decimal:
    scaled = (dec * Decimal(100)).to_integral_value(rounding=ROUND_FLOOR)
    return Decimal(scaled) / Decimal(100)


def parse_number_text_to_money_str(s: str) -> str | None:
    """
    網頁 span 可能是 '21,321,123.12' 或 '0.00'
    解析後回傳 '21321123.12'（並截斷到小數點後兩位）
    """
    if not s:
        return None
    s = str(s).strip().replace(",", "")
    m = re.search(r"-?\d+(?:\.\d+)?", s)
    if not m:
        return None
    try:
        dec = Decimal(m.group(0))
        trunc = _truncate_2_from_decimal(dec)
        return f"{trunc:.2f}"
    except Exception:
        return None


def wait_page_ready(wait: WebDriverWait):
    wait.until(EC.presence_of_element_located((By.ID, "stock")))


def fill_int_by_id(driver, input_id: str, value: int):
    el = driver.find_element(By.ID, input_id)
    safe_clear(driver, el)
    human_type(el, str(value), delay=TYPE_DELAY)


# =========================================================
# ✅ 依 input_id 找「同列」button + 同列 text-danger
# =========================================================
def click_calc_and_read_by_input_id(driver, input_id: str, timeout=10.0) -> str | None:
    """
    依你貼的 HTML 結構：
      - 每列有自己的 button[title=計算]
      - 結果在同一個 d-flex align-items-center 內的 span.text-danger
    """
    def locate():
        inp = driver.find_element(By.ID, input_id)

        row = inp.find_element(
            By.XPATH,
            "ancestor::div[contains(@class,'form-group') and contains(@class,'row')][1]"
        )

        btn = row.find_element(By.CSS_SELECTOR, "button[title='計算']")

        dflex = btn.find_element(
            By.XPATH,
            "ancestor::div[contains(@class,'d-flex') and contains(@class,'align-items-center')][1]"
        )
        span = dflex.find_element(By.CSS_SELECTOR, "span.text-danger")
        return btn, span

    btn, span = locate()
    before = (span.text or "").strip()

    safe_click(driver, btn)

    end = time.time() + timeout
    last_text = before

    while time.time() < end:
        try:
            _, span2 = locate()
            cur = (span2.text or "").strip()
            last_text = cur

            money = parse_number_text_to_money_str(cur)
            if money is not None:
                if cur != before:
                    return money
                # 若之前不是 0.00（殘值），也希望等到有效值
                if before != "0.00" and cur != "0.00":
                    return money

        except StaleElementReferenceException:
            pass

        time.sleep(0.10)

    return parse_number_text_to_money_str(last_text)


# =========================================================
# 隨機參數
# =========================================================
def rand_tp():
    return random.randint(1000, 1_000_000)

def rand_ep():
    return random.randint(10, 100)

def rand_sc():
    return random.randint(100, 10_000)

def rand_ca():
    return random.randint(1_000_000, 50_000_000)

def rand_pa():
    return random.randint(10_000, 200_000)


# =========================================================
# 固定欄位 id（依你貼的 HTML）
# =========================================================
SEC_CASES = [
    ("stock", "stock"),                               # (一)
    ("bond", "securities"),                           # (二)
    ("warrant", "warrants"),                          # (三)
    ("warrant_delivery_stock", ("price1", "count1")), # (四)-1 股票交割
    ("warrant_delivery_cash", ("price2", "count2")),  # (四)-2 現金結算
]

FUT_CASES = [
    ("stock_index", "transactionTax1"),
    ("interest_rate_30", "transactionTax2"),
    ("interest_rate_10", "transactionTax3"),
    ("option", "transactionTax4"),  # 權利金額
    ("gold", "transactionTax5"),
]


def run_securities_tests(driver, wait: WebDriverWait, n=50):
    calc_sec, _ = _get_local_functions()

    print("===== 三、證券交易稅：本地 vs 網頁 =====")
    mismatches = 0

    for i in range(n):
        tax_item, payload = random.choice(SEC_CASES)

        tp = ep = sc = None

        if tax_item in ("stock", "bond", "warrant"):
            input_id = payload
            tp = rand_tp()

            fill_int_by_id(driver, input_id, tp)

            # ✅ 直接呼叫你 notebook 已定義好的本地函式
            local = calc_sec(tax_item, tp)
            web = click_calc_and_read_by_input_id(driver, input_id, timeout=10.0)

        else:
            price_id, count_id = payload
            ep = rand_ep()
            sc = rand_sc()

            fill_int_by_id(driver, price_id, ep)
            fill_int_by_id(driver, count_id, sc)

            # ✅ 你的本地函式簽名需要 tp，但此項不使用 tp：塞 0
            local = calc_sec(tax_item, 0, ep=ep, sc=sc)
            web = click_calc_and_read_by_input_id(driver, count_id, timeout=10.0)

        ok = (web is not None) and (local == web)
        if not ok:
            mismatches += 1
            print(f"[{i+1:03d}] ❌ mismatch | item={tax_item}")
            print(f"  tp={tp}, ep={ep}, sc={sc}")
            print(f"  local={local}, web={web}")
        else:
            print(f"[{i+1:03d}] ✅ ok | item={tax_item} | local=web={local}")

        driver.refresh()
        time.sleep(1.1)
        wait_page_ready(wait)

    print(f"證券交易稅：總共 {n} 筆，mismatch={mismatches}\n")


def run_futures_tests(driver, wait: WebDriverWait, n=50):
    _, calc_fut = _get_local_functions()

    print("===== 四、期貨交易稅：本地 vs 網頁 =====")
    mismatches = 0

    for i in range(n):
        tax_item, input_id = random.choice(FUT_CASES)

        ca = pa = None

        if tax_item == "option":
            pa = rand_pa()
            fill_int_by_id(driver, input_id, pa)

            # ✅ option 用 pa 計算：ca 不用，塞 0
            local = calc_fut("option", 0, pa=pa)
            web = click_calc_and_read_by_input_id(driver, input_id, timeout=10.0)

        else:
            ca = rand_ca()
            fill_int_by_id(driver, input_id, ca)

            local = calc_fut(tax_item, ca)
            web = click_calc_and_read_by_input_id(driver, input_id, timeout=10.0)

        ok = (web is not None) and (local == web)
        if not ok:
            mismatches += 1
            print(f"[{i+1:03d}] ❌ mismatch | item={tax_item}")
            print(f"  ca={ca}, pa={pa}")
            print(f"  local={local}, web={web}")
        else:
            print(f"[{i+1:03d}] ✅ ok | item={tax_item} | local=web={local}")

        driver.refresh()
        time.sleep(1.1)
        wait_page_ready(wait)

    print(f"期貨交易稅：總共 {n} 筆，mismatch={mismatches}\n")


def run_all(seed=42, securities_n=50, futures_n=50):
    random.seed(seed)

    driver = webdriver.Chrome()
    wait = WebDriverWait(driver, 20)

    driver.delete_all_cookies()
    driver.get(URL)
    time.sleep(2)
    wait_page_ready(wait)

    try:
        run_securities_tests(driver, wait, n=securities_n)
        run_futures_tests(driver, wait, n=futures_n)
    finally:
        driver.quit()


if __name__ == "__main__":
    run_all(seed=42, securities_n=50, futures_n=50)


===== 三、證券交易稅：本地 vs 網頁 =====
[001] ✅ ok | item=stock | local=web=81.67
[002] ✅ ok | item=warrant | local=web=257.78
[003] ✅ ok | item=bond | local=web=147.31
[004] ✅ ok | item=stock | local=web=2131.71
[005] ✅ ok | item=warrant_delivery_cash | local=web=205.25
[006] ✅ ok | item=warrant_delivery_stock | local=web=24.69
[007] ✅ ok | item=stock | local=web=690.77
[008] ✅ ok | item=bond | local=web=530.90
[009] ✅ ok | item=warrant_delivery_cash | local=web=120.83
[010] ✅ ok | item=bond | local=web=751.80
[011] ✅ ok | item=warrant_delivery_cash | local=web=233.79
[012] ✅ ok | item=warrant_delivery_stock | local=web=1187.53
[013] ✅ ok | item=stock | local=web=2390.00
[014] ✅ ok | item=bond | local=web=733.05
[015] ✅ ok | item=warrant_delivery_stock | local=web=739.66
[016] ✅ ok | item=bond | local=web=226.77
[017] ✅ ok | item=warrant | local=web=108.17
[018] ✅ ok | item=stock | local=web=1198.14
[019] ✅ ok | item=stock | local=web=1132.25
[020] ✅ ok | item=warrant | local=web=634.05
[021] ✅ 

### According to the selenium test above, we have 0 mismatches against the MoF protal